# Import Requirements

In [1]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
import random
import json

In [2]:
# --- CONFIGURATION ---
EPOCHS = 30                # Number of training loops
BATCH_SIZE = 8             # Process 8 sentences at a time
DROPOUT = 0.5              # Randomness to prevent overfitting
MODEL_NAME = "en_core_web_md" # The base model we are fine-tuning

# 1. LOAD THE DATA

In [3]:
# We load the JSON file generated in the training_data.py
print("Loading dataset...")
try:
    with open("train_financial_ner.json", "r") as f:
        TRAIN_DATA = json.load(f)
    print(f"Loaded {len(TRAIN_DATA)} training examples.")
except FileNotFoundError:
    print("Error: 'train_financial_ner.json' not found. Please run the Data Generation script first!")
    exit()

Loading dataset...
Loaded 294 training examples.


# 2. LOAD THE BASE MODEL

In [4]:
# We load the medium English model (must install first: python -m spacy download en_core_web_md)
nlp = spacy.load(MODEL_NAME)

# 3. SETUP THE PIPELINE

In [5]:
# Check if NER pipe exists, if not add it
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner")
else:
    ner = nlp.get_pipe("ner")

In [6]:
# 4. ADD NEW LABELS
# The video loops through data to find labels. We do the same.
print("Adding labels to model...")
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

Adding labels to model...


# 5. DISABLE OTHER PIPELINES

In [7]:
# We only want to train NER, not the tagger or parser
pipe_exceptions = ["ner", "trf_tok2vec", "tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# 6. TRAINING LOOP

In [8]:
print(f"Starting training for {EPOCHS} epochs...")
with nlp.disable_pipes(*other_pipes):
    
    # Initialize optimizer
    optimizer = nlp.create_optimizer()
    
    for epoch in range(EPOCHS):
        random.shuffle(TRAIN_DATA)
        losses = {}
        
        # Create batches
        # We use compounding to start with small batches and increase them
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        
        for batch in batches:
            texts, annotations = zip(*batch)
            
            # Convert to spaCy 'Example' objects (Required for spaCy v3)
            examples = []
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example = Example.from_dict(doc, annotations[i])
                examples.append(example)
            
            # Update the model
            nlp.update(
                examples,
                drop=DROPOUT,
                losses=losses,
                sgd=optimizer
            )
            
        print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {losses['ner']:.2f}")

Starting training for 30 epochs...
Epoch 1/30 - Loss: 1009.74
Epoch 2/30 - Loss: 705.48
Epoch 3/30 - Loss: 616.31
Epoch 4/30 - Loss: 544.89
Epoch 5/30 - Loss: 502.09
Epoch 6/30 - Loss: 479.55
Epoch 7/30 - Loss: 448.28
Epoch 8/30 - Loss: 419.45
Epoch 9/30 - Loss: 415.54
Epoch 10/30 - Loss: 418.23
Epoch 11/30 - Loss: 392.82
Epoch 12/30 - Loss: 371.98
Epoch 13/30 - Loss: 408.09
Epoch 14/30 - Loss: 343.58
Epoch 15/30 - Loss: 321.66
Epoch 16/30 - Loss: 323.94
Epoch 17/30 - Loss: 286.59
Epoch 18/30 - Loss: 271.66
Epoch 19/30 - Loss: 286.69
Epoch 20/30 - Loss: 264.65
Epoch 21/30 - Loss: 283.77
Epoch 22/30 - Loss: 264.30
Epoch 23/30 - Loss: 234.17
Epoch 24/30 - Loss: 258.32
Epoch 25/30 - Loss: 244.17
Epoch 26/30 - Loss: 215.37
Epoch 27/30 - Loss: 201.50
Epoch 28/30 - Loss: 199.02
Epoch 29/30 - Loss: 198.20
Epoch 30/30 - Loss: 220.11


# 7. SAVE THE MODEL

In [9]:
output_dir = "financial_ner_model"
nlp.to_disk(output_dir)
print(f"\nModel saved to directory: {output_dir}")


Model saved to directory: financial_ner_model


# 8. TEST THE MODEL (Immediate Sanity Check)

In [10]:
print("\n--- TESTING THE NEW MODEL ---")
test_sentences = [
    "Apple (AAPL) announced the acquisition of an AI startup for $50 million.",
    "The CPI rose 0.4% in March, signaling inflation.",
    "Nvidia announced a stock split after hitting $900."
]

for text in test_sentences:
    doc = nlp(text)
    print(f"\nText: {text}")
    for ent in doc.ents:
        print(f"  {ent.text} -> {ent.label_}")


--- TESTING THE NEW MODEL ---

Text: Apple (AAPL) announced the acquisition of an AI startup for $50 million.
  Apple -> ORG
  AAPL -> TICKER
  acquisition -> EVENT
  $50 million -> MONEY

Text: The CPI rose 0.4% in March, signaling inflation.
  0.4% -> PERCENT
  March -> DATE

Text: Nvidia announced a stock split after hitting $900.
  Nvidia -> ORG
  stock split -> EVENT
  $900 -> MONEY


# Model Evaluation

In [11]:
!python evaluate_model.py

Loading model from: financial_ner_model...
Loading validation data from: dev_financial_ner.json...
Evaluating on 66 examples...

METRIC          | SCORE     
Precision       | 77.40%
Recall          | 82.56%
F1-Score        | 79.90%

Breakdown by Entity Type:
ENTITY       | PRECISION  | RECALL     | F1        
------------------------------------------------
DATE         | 90.00%    | 100.00%    | 94.74%
EVENT        | 57.14%    | 44.44%    | 50.00%
INDICATOR    | 10.00%    | 20.00%    | 13.33%
MONEY        | 85.71%    | 96.00%    | 90.57%
ORG          | 78.49%    | 85.88%    | 82.02%
PER          | 71.43%    | 90.91%    | 80.00%
PERCENT      | 93.75%    | 93.75%    | 93.75%
ROLE         | 85.71%    | 100.00%    | 92.31%
TICKER       | 76.92%    | 50.00%    | 60.61%


# Update Training Data

In [12]:
!python update_dataset.py

Loaded 294 existing examples.
Success! Added 30 new repair examples.
Total Training Dataset Size: 324


# Evaluate model after updating Training Data

In [13]:
!python evaluate_model.py

Loading model from: financial_ner_model...
Loading validation data from: dev_financial_ner.json...
Evaluating on 66 examples...

METRIC          | SCORE     
Precision       | 77.40%
Recall          | 82.56%
F1-Score        | 79.90%

Breakdown by Entity Type:
ENTITY       | PRECISION  | RECALL     | F1        
------------------------------------------------
DATE         | 90.00%    | 100.00%    | 94.74%
EVENT        | 57.14%    | 44.44%    | 50.00%
INDICATOR    | 10.00%    | 20.00%    | 13.33%
MONEY        | 85.71%    | 96.00%    | 90.57%
ORG          | 78.49%    | 85.88%    | 82.02%
PER          | 71.43%    | 90.91%    | 80.00%
PERCENT      | 93.75%    | 93.75%    | 93.75%
ROLE         | 85.71%    | 100.00%    | 92.31%
TICKER       | 76.92%    | 50.00%    | 60.61%


# Update Validation Data

In [14]:
!python update_dev.py

Loaded 66 existing Dev examples.
Success! Added 15 new examples to the Dev Set.
New Dev Set Size: 81


# Evaluate on updated Validation Data

In [15]:
!python evaluate_model.py

Loading model from: financial_ner_model...
Loading validation data from: dev_financial_ner.json...
Evaluating on 81 examples...

METRIC          | SCORE     
Precision       | 75.31%
Recall          | 82.19%
F1-Score        | 78.60%

Breakdown by Entity Type:
ENTITY       | PRECISION  | RECALL     | F1        
------------------------------------------------
DATE         | 85.71%    | 100.00%    | 92.31%
EVENT        | 71.43%    | 58.82%    | 64.52%
INDICATOR    | 31.25%    | 41.67%    | 35.71%
MONEY        | 77.42%    | 96.00%    | 85.71%
ORG          | 79.61%    | 87.23%    | 83.25%
PER          | 71.43%    | 90.91%    | 80.00%
PERCENT      | 75.00%    | 93.75%    | 83.33%
ROLE         | 85.71%    | 100.00%    | 92.31%
TICKER       | 76.92%    | 50.00%    | 60.61%


# Model is Ready to use on real text!! 🎉🗿